In [2]:
import pandas as pd, numpy as np
import os, sys, path
import IPython.display as ipd
import torchaudio
from sklearn.model_selection import train_test_split

C:\Users\Narahari\anaconda3\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [3]:
torchaudio.get_audio_backend()

'soundfile'

In [4]:
parent_dir = path.Path(os.getcwd()).parent

print(parent_dir)

D:\Personal\future\speech


In [21]:
pd.options.display.max_colwidth = 500

In [69]:
#------ Input Files----#

raw_dir = os.path.join(parent_dir, "data", "raw")

print(raw_dir)

transcribed_files = [os.path.join(raw_dir, "kn_in_female", "line_index.tsv"),
                     os.path.join(raw_dir, "kn_in_male", "line_index.tsv")]

#----- Output Files-----#

trans_map_file = os.path.join(parent_dir, "data", "processed", "transcribed_kannada_abs_fpath.tsv")

train_openslr_fname = os.path.join(parent_dir, "data", "model_inputs", "train_openslr.tsv")

test_openslr_fname = os.path.join(parent_dir, "data", "model_inputs", "test_openslr.tsv")

#--- Parameters-------#

test_ratio = 0.2

random_state = 121

D:\Personal\future\speech\data\raw


In [10]:
path.Path(raw_dir).parent

Path('D:\\Personal\\future\\speech\\data')

In [ ]:
def unzip_data(fname, dest_folder):

    with zipfile.ZipFile(fname, 'r') as zip_ref:

        zip_ref.extractall(dest_folder)

def download_and_extract(name="openslr_kannada", out_folder = ""):
    if name=="openslr_kannada":

        url_list = ["https://www.openslr.org/resources/79/kn_in_male.zip", "https://www.openslr.org/resources/79/kn_in_female.zip"]

        out_paths = [os.path.join(out_folder, "kn_in_male.zip"), os.path.join(out_folder, "kn_in_female.zip")]

        unzip_paths = [os.path.join(out_folder, "kn_in_male"), os.path.join(out_folder, "kn_in_female")]

        url_paths = zip(url_list, out_paths, unzip_paths)

    for u, p, uz in url_paths:
        
        print("Downloading ", u, "to", p)
        urllib.request.urlretrieve(u, p)
        print("Unzipping to:", uz)
        unzip_data(p, uz)

download_and_extract(name="openslr_kannada", out_folder = raw_dir)

In [42]:
def prepare_trans_map_file(t_files, trans_map_file=None, audio_ext=".wav", sep="\t"):
    """
    Combines all transciption files and adds absolute path
    """
    t_map_df = pd.DataFrame()
    
    for fname in t_files:
        
        p_fn = path.Path(fname)
        
        parent_fn = p_fn.parent
        
        map_df = pd.read_csv(fname, sep=sep, encoding="utf-8", header=None, names = ["path", "sentence"])
        
        map_df["path"] = map_df["path"].apply(lambda x: os.path.join(parent_fn, x + audio_ext))
        
        #print(map_df.sample(5))
        
        t_map_df = pd.concat([t_map_df, map_df], ignore_index=True)
    
    if trans_map_file != None:
        
        print("Writing trans map file to:", trans_map_file)
        
        t_map_df.to_csv(trans_map_file, sep="\t", encoding="utf-8", index=False)
    
    return t_map_df

trans_df = prepare_trans_map_file(transcribed_files, trans_map_file=trans_map_file)

trans_df

Writing trans map file to: D:\Personal\future\speech\data\processed\transcribed_kannada_abs_fpath.tsv


,path,sentence
0,D:\Personal\future\speech\data\raw\kn_in_female\knf_00557_01097230618.wav,ಪರ್ವತಗಳು ವಾಯುವಿಗನುಗುಣವಾಗಿ ಕಂಪಿಸುವುವು.
1,D:\Personal\future\speech\data\raw\kn_in_female\knf_03271_01629528503.wav,ಚಿತ್ರವನ್ನು ಕೆಲವೊಮ್ಮೆ ವಾರ್ನಿಷ್ ಅಥವಾ ಕೆಲವು ಇತರ ಮುದ್ರಕಗಳನ್ನು ರಕ್ಷಣೆಯಾಗಿ ಲೇಪನ ಮಾಡಲಾಗುತ್ತದೆ.
2,D:\Personal\future\speech\data\raw\kn_in_female\knf_03574_00145342156.wav,ಹಚ್ಚುವ ಮೊದಲು ಲೋಹಗಳ ಮೈಯನ್ನು ಉಜ್ಜಿ ಒರಟು ಮಾಡುವುದು ಒಳ್ಳೆಯದು.
3,D:\Personal\future\speech\data\raw\kn_in_female\knf_00557_01945435360.wav,ಹೂಗೊಂಚಲಿನಲ್ಲಿ ಹೆಣ್ಣು ಹೂಗಳು ಮಾತ್ರವಿದೆ
4,D:\Personal\future\speech\data\raw\kn_in_female\knf_02173_02031667415.wav,ನಾಲ್ನೂರು ಗಜಗಳಲ್ಲಿ
...,...,...
4395,D:\Personal\future\speech\data\raw\kn_in_male\knm_09877_00045560368.wav,ಈತ 1835ರಲ್ಲಿ ಮೆಡ್ಡಗಣ್ಣಿನೊಂದಿಗಿನ ಗಳಗಂಡದ ಒಂದು ಪ್ರಕರಣವನ್ನು ವಿವರಿಸಿದ.
4396,D:\Personal\future\speech\data\raw\kn_in_male\knm_00417_00907639954.wav,ದೋಣಿಯಲ್ಲಿ ಪ್ರಯಾಣಿಸಿ
4397,D:\Personal\future\speech\data\raw\kn_in_male\knm_00180_00827448517.wav,"ನಂತರ ಈ ಅಂದಾಜಿನ ಪ್ರದೇಶಗಳಿಗೆ ಸಂಬಂಧಿಸಿದಂತೆ ಅಂಕಿ, ಅಂಶೀಯ ಜನಗಣತಿ ದತ್ತಾಂಶವನ್ನು ಒದಗಿಸಲಾಗುತ್ತದೆ."
4398,D:\Personal\future\speech\data\raw\kn_in_male\knm_00574_00356339818.wav,ಅಂತರಾಷ್ಟ್ರೀಯ ವ್ಯವಹಾರವು ಎರಡು ಅಥವಾ ಹಲವು ದೇಶಗಳ ನಡುವೆ ನಡೆಯುವ ವ್ಯವಹಾರವಾಗಿದೆ


##### Getting familiar with data

In [63]:
trans_df.sample(10)

,path,sentence
957,D:\Personal\future\speech\data\raw\kn_in_female\knf_09123_01153985744.wav,ಸಾಮಾನ್ಯವಾಗಿ ಅಂತರರಾಷ್ಟ್ರೀಯ ಹಣಕಾಸು ನಿಧಿ ಹಾಗೂ ಅದರ ಬೆಂಬಲಿಗರು ವಿತ್ತೀಯ ನಿಯಂತ್ರಣವನ್ನು ಆಚರಣೆಗೆ ತರಬೇಕೆಂದು ಹೇಳುತ್ತಾರೆ
3679,D:\Personal\future\speech\data\raw\kn_in_male\knm_00271_01832191140.wav,ಅದು ಅಂತಾರಾಷ್ಟ್ರೀಯ ಸಂಸ್ಥೆಗಳ ಉಗಮಕ್ಕೆ ಅವಕಾಶ ಮಾಡಿಕೊಟ್ಟಿತು.
2100,D:\Personal\future\speech\data\raw\kn_in_female\knf_01493_00910272760.wav,ಮುದ್ರಿತವಾಗಿರುವ ಅಂಚೆಯ ಚೀಟಿಗಳನ್ನೊಳಗೊಂಡ ಕಾರ್ಡು ಮತ್ತು ಲಕೋಟೆಗಳು ಎಲ್ಲ ಅಂಚೆ ಕಚೇರಿಗಳಲ್ಲೂ ಮಾರಾಟಕ್ಕಿರುತ್ತವೆ.
1875,D:\Personal\future\speech\data\raw\kn_in_female\knf_03755_02100068874.wav,ಈ ಕಾಯಿದೆಗೆ ವಿವಿಧ ರಾಷ್ಟ್ರಗಳ ನಡುವೆ ಆಗಿರುವ ಒಪ್ಪಂದವೇ ಆಧಾರವಾಗಿದೆ.
2181,D:\Personal\future\speech\data\raw\kn_in_female\knf_01493_00549895262.wav,ಮುಂದಿನ ಬೇಸಗೆಯಲ್ಲಿ ಫಸಲು ಬರುತ್ತದೆ
4231,D:\Personal\future\speech\data\raw\kn_in_male\knm_00811_00487051432.wav,ಹನ್ನೆರಡು ಇಂಚುಗಳಿಗೆ ಒಂದು ಅಡಿ
4133,D:\Personal\future\speech\data\raw\kn_in_male\knm_04526_00748085712.wav,ಬೇಸಿಗೆಯಲ್ಲಿ ತೀರ ಪ್ರದೇಶದಲ್ಲಿ ಉಷ್ಣತೆ ಸೊನ್ನೆ ಡಿಗ್ರಿ ಸೆಂಟಿಗ್ರೇಡ್ ಗೆ ಇಳಿದರೆ
1236,D:\Personal\future\speech\data\raw\kn_in_female\knf_03755_02077303565.wav,ಆದರೆ ಯುದ್ಧಕಾಲದಲ್ಲಿ ಇದರಲ್ಲಿ ಒಡಕು ಹುಟ್ಟಿದ್ದರಿಂದ 1919ರಲ್ಲಿ ಅಂತಾರಾಷ್ಟ್ರೀಯ III ರ ನಿರ್ಮಾಣ ಅಗತ್ಯವಾಯಿತು.
3050,D:\Personal\future\speech\data\raw\kn_in_male\knm_09140_01851490655.wav,ಇದು ಪ್ರತ್ಯೇಕ ಜಲರಾಶಿಯಲ್ಲ.
3563,D:\Personal\future\speech\data\raw\kn_in_male\knm_01114_01454056466.wav,ಉತ್ಪಾದನೀಯ ಅಂಶಗಳನ್ನು ಆಮದುಮಾಡಿಕೊಳ್ಳುವುದರ ಬದಲು ವಿನಿಮಯ ಮಾಡಿಕೊಳ್ಳಬಹುದು


In [60]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

sample_df = trans_df.loc[[3055, 3056, 3057,3058, 4124, 4125], :].apply(speech_file_to_array_fn, axis=1)

sample_df

,path,sentence,speech,sampling_rate,target_text
3055,D:\Personal\future\speech\data\raw\kn_in_male\knm_05927_00880807200.wav,"ಹೂಬಿಡುವ ಕಾಲ ಅಕ್ಟೋಬರ್, ನವೆಂಬರ್.","[-9.1552734e-05, -6.1035156e-05, -3.0517578e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0517578e-05, 3.0517578e-05, 3.0517578e-05, 3.0517578e-05, 3.0517578e-05, 3.0517578e-05, 3....",48000,"ಹೂಬಿಡುವ ಕಾಲ ಅಕ್ಟೋಬರ್, ನವೆಂಬರ್."
3056,D:\Personal\future\speech\data\raw\kn_in_male\knm_00271_00416120252.wav,ಎಡಕ್ಕೆ ತಿರುಗಿ,"[0.0, 3.0517578e-05, 0.0, 0.0, -3.0517578e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -9.1552734e-05, -9.1552734e-05, -0.00012207031, -0.00012207031, -0.00012207031, -0.00012207031, -9.1552734e-05, -9.1552734e-05, -0.00012207031, -0.00015258789, -0.00018310547, -0.00021362305, -0.00021362305, -0.00021362305, -0.00021362305, -0.00021362305, -0.00021362305, -0.00021362305, -0.00021362305, -0.00021362305, -0.00021362305, -...",48000,ಎಡಕ್ಕೆ ತಿರುಗಿ
3057,D:\Personal\future\speech\data\raw\kn_in_male\knm_00417_01646795202.wav,ಸರಿಯಾದ ಎಡಕ್ಕೆ,"[-3.0517578e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -3.0517578e-05, -3.0517578e-05, -3.0517578e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -3.0517578e-05, -3.0517578e-05, -3.0517578e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -3.0517578e-05, 0.0, 0.0, 0.0, -3.0517578e-05, -6.1035156e-05, -3.0517578e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.0517578e-05, -3.0517578e-05, -3.0517578e-05, 0.0, 0.0, 0.0, 0.0, 0...",48000,ಸರಿಯಾದ ಎಡಕ್ಕೆ
3058,D:\Personal\future\speech\data\raw\kn_in_male\knm_01672_00405778876.wav,ಅಂದರೆ ಬೇಕೆನಿಸಿದಾಗ ಸರಕುಗಳು ಲಭಿಸುವುದಿಲ್ಲ.,"[-6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05, -6.1035156e-05,...",48000,ಅಂದರೆ ಬೇಕೆನಿಸಿದಾಗ ಸರಕುಗಳು ಲಭಿಸುವುದಿಲ್ಲ.
4124,D:\Personal\future\speech\data\raw\kn_in_male\knm_07722_01781030045.wav,ಮೂರನೆಯದು ಸ್ಮಿರ್ನ ಅಂಜೂರ,"[-0.00015258789, -0.00015258789, -0.00015258789, -0.00015258789, -0.00015258789, -0.00015258789, -0.00015258789, -0.00018310547, -0.00018310547, -0.00015258789, -0.00015258789, -0.00015258789, -0.00018310547, -0.00018310547, -0.00018310547, -0.00018310547, -0.00015258789, -0.00012207031, -9.1552734e-05, -6.1035156e-05, -6.1035156e-05, -9.1552734e-05, -9.1552734e-05, -9.1552734e-05, -9.1552734e-05, -6.1035156e-05, -6.1035156e-05, -9.1552734e-05, -9.1552734e-05, -9.1552734e-05, -6.1035156e-05,...",48000,ಮೂರನೆಯದು ಸ್ಮಿರ್ನ ಅಂಜೂರ
4125,D:\Personal\future\speech\data\raw\kn_in_male\knm_00271_00079684796.wav,", ನಂತರ","[0.00033569336, 0.00030517578, 0.0002746582, 0.00021362305, 0.00018310547, 0.00018310547, 0.00018310547, 0.00018310547, 0.00018310547, 0.00015258789, 0.00012207031, 9.1552734e-05, 6.1035156e-05, 6.1035156e-05, 6.1035156e-05, 9.1552734e-05, 0.00012207031, 0.00012207031, 0.00012207031, 0.00012207031, 9.1552734e-05, 6.1035156e-05, 6.1035156e-05, 3.0517578e-05, 3.0517578e-05, 6.1035156e-05, 6.1035156e-05, 3.0517578e-05, 3.0517578e-05, 3.0517578e-05, 3.0517578e-05, 3.0517578e-05, 3.0517578e-05, 3...",48000,", ನಂತರ"


In [62]:
ipd.Audio(data=np.asarray(sample_df.loc[4125, "speech"]), autoplay=True, rate=48000)

In [73]:
def create_train_test(fname, train_fname, test_fname, split = 0.2, random_state = 121):
    
    full_df = pd.read_csv(fname, sep="\t",  encoding="utf-8")
    
    train, test = train_test_split(full_df, test_size=split, random_state=random_state )
    
    test.to_csv(test_fname, encoding="utf-8", index=False, sep="\t")
    
    train.to_csv(train_fname, encoding="utf-8", index=False, sep="\t")
    
    print("Writing train-test to:", train_fname, test_fname)
    
    return train, test

train_df, test_df = create_train_test( trans_map_file, train_openslr_fname, test_openslr_fname,  
                                      split = test_ratio, random_state = random_state)

print(train_df.shape, test_df.shape)

Writing train-test to: D:\Personal\future\speech\data\model_inputs\train_openslr.tsv D:\Personal\future\speech\data\model_inputs\test_openslr.tsv
(3520, 2) (880, 2)
